In [1]:
import os
import uuid
from openai import OpenAI
from dotenv import load_dotenv
from flask import Flask, request, jsonify, render_template

In [2]:
load_dotenv()
OPEN_AI_API_KEY = os.getenv("API_KEY")
client = OpenAI(api_key = OPEN_AI_API_KEY)

In [3]:
app = Flask(__name__)

In [4]:
documents = {}
test_answers = {}

In [5]:
@app.route("/")
def index():
    return render_template("upload.html")

@app.route("/upload", methods=['POST'])
def upload():
    if "files" not in request.files:
        return jsonify({"Error": "No file uploaded"}), 400

    file = request.files["files"]
    document_id = str(uuid.uuid4())

    try:
        # Read the file content and decode it (consider using an in-memory buffer for larger files)
        document_text = file.read().decode("ISO-8859-1")  # Adjust encoding as needed
        file.close()

        # Call the helper function to summarize the document using Assistant
        summary = summarize_file(document_text)
        return jsonify({"document_id": document_id}), 200
    except Exception as e:
        return jsonify({"Error": f"An error occurred: {e}"}), 500


def summarize_file(doc):
    try:
        # Create OpenAI file object (consider using a temporary file for large documents)
        with OpenAI.File.create(content=doc, purpose="assistants") as file_obj:
            # Create assistant with file access
            assistant = OpenAI.Assistant.create(
                name="Data Summarizer",
                description="You are great at summarizing data from files. You also share a brief text summary of the file.",
                model="gpt-3.5-turbo-1106",  # Adjust model as needed
                file_ids=[file_obj.id],
                tasks=[
                    OpenAI.Task.complete(
                        prompt=f"Summarize the following document:\n{doc}",
                        max_tokens=150,  # Adjust number of tokens for summary length
                        n=1,
                        stop=None,
                        temperature=0.7,
                    )
                ],
            )

            # Extract and return the summary
            summary = assistant.response.choices[0].text.strip()
            return summary
    except Exception as e:
        return jsonify({"Error": f"Error generating summary: {e}"}), 500

    
def Gen_answer(document_text):
    try:
        response = client.beta.assistants.create(  # Remove the 'messages' argument
            model="gpt-3.5-turbo-1106",
            content=f"Generate a test answer for the following document:\n{document_text}"
        )
        return response
    except Exception as e:
        return jsonify({"Error": "Error generating answer from OpenAI API: {}".format(e)}), 500

# """
# def Gen_answer(document_text):
#     try:
#         response = client.beta.assistants.create(
#             model="gpt-3.5-turbo-1106",
#             messages=[
#                 {"role": "system", "content": "You are my research assistant."},
#                 {"role": "user", "content": f"Generate a test answer for the following document:\n{document_text}"}
#             ]
#         )
#         return response
#     except Exception as e:
#         return jsonify({"Error": "Error generating answer from OpenAI API: {}".format(e)}), 500


In [6]:
if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Mar/2024 09:27:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2024 09:27:28] "POST /upload HTTP/1.1" 500 -


In [ ]:
#Endpoint to many the Q>A>T session
@app.route('/query/<string:document_id>/', methods=['POST'])
def query_document(document_id):
    if document_id not in documents:
        return jsonify({"error": "Document not found"}), 404

    data = request.get_json()
    question = data.get("question")

    if not question:
        return jsonify({"error": "Question not provided"}), 400
    
    #Generate test Question
    test_questions_id = str(uuid.uuid4)
    test_question = generate_test_question(question)

    # Simulate a test answer retrieval from the database
    #test_answer = test_answers.get(document_id, "") 

    response = {
        "answer": generate_answer(question, documents[document_id]),
        "bullet_points": ["Point 1", "Point 2", "Point 3"],
        "test_question": test_question,
        "test_question_id": test_questions_id,
    }

    return jsonify(response)

# Generate a test question using OpenAI Assistant API
def generate_test_question(question):
    # Use OpenAI Assistant API to generate a test question based on the user's question
    response = assistant.append_message(
        model="text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Generate a test question for the following user question:\n{question}"}
        ]
    )
    
    test_question_content = response['choices'][0]['message']['content']
    
    return test_question_content

# Generate an answer using OpenAI Assistant API
def generate_answer(question, document_text):
    # Use OpenAI Assistant API to generate an answer based on the user's question and document
    response = assistant.append_message(
        model="text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{question}\nDocument: {document_text}"}
        ]
    )
    
    answer_content = response['choices'][0]['message']['content']
    
    return answer_content

    